ModuleNotFoundError: No module named 'geopy'

In [2]:
pip install geopy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.2 MB/s eta 0:00:001 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.7 MB/s eta 0:00:000:00:01 eta 0:00:01
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need 

In [3]:
import json
from geopy import distance
from datetime import datetime
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


def speed_location(point_list, point):
    if point not in point_list:
        point_list.append([point, 1])
    for i in range(len(point_list)):
        if distance.distance(point_list[i][0], point) < 0.5 and point_list[i][0] != point:
            point_list[i][1] += 1
    return point_list


def calculate_time(bus_list, index):
    format = '%Y-%m-%d %H:%M:%S'
    time1 = datetime.strptime(bus_list[index - 1].get('Time'), format)
    time2 = datetime.strptime(bus_list[index].get('Time'), format)
    time = time2 - time1
    time = time.total_seconds() / 3600
    return time


def position(bus_list, index):
    pos = (bus_list[index].get('Lat'), bus_list[index].get('Lon'))
    return pos


def calculate_distance(bus_list, index):
    pos1 = position(bus_list, index - 1)
    pos2 = position(bus_list, index)
    dis = distance.distance(pos1, pos2).km
    return dis


def number_of_buses(data):
    counter = 0
    for bus_list in data:
        counter += 1
    return counter


def bus_speed_check(filename):
    point_list = []
    speed_counter = 0
    with open(filename, 'r') as file:
        data = json.load(file)
    for bus_list in data:
        speeding = False
        for j in range(1, len(bus_list)):
            dis = calculate_distance(bus_list, j)
            time = calculate_time(bus_list, j)
            if time != 0:
                vel = dis / time
            else:
                vel = 0
            if vel > 50:
                speeding = True
                point_list = speed_location(point_list, position(bus_list, j))
            # print(bus_list[j].get('Lines'), ': ', vel)
        if speeding:
            speed_counter += 1
    return speed_counter, point_list



filename = 'bus_positions_divided.json'
filename2 = 'bus_positions_divided2.json'
tup1 = bus_speed_check(filename)
tup2 = bus_speed_check(filename2)

speeding1 = tup1[0]
point_list1 = tup1[1]
speeding2 = tup2[0]
point_list2 = tup2[1]

plt.interactive(False)
x = ['13:00', '16:00']
y = [speeding1, speeding2]
plt.bar(x, y)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'bus_positions_divided.json'